In [10]:
from collections import OrderedDict
import os
import glob
import numpy as np
import pandas as pd
import dask
from dask.delayed import delayed
from dask.distributed import Client, wait
import dask.dataframe
import shutil

In [2]:
use_local_cuda_cluster = False
# Use below for a local-only CUDA cluster
if use_local_cuda_cluster:
    from dask_cuda import LocalCUDACluster
    cluster = LocalCUDACluster(ip='0.0.0.0')
    client = Client(cluster)

In [3]:
# Use below for a multi-host multi-GPU CUDA cluster started with start_dask.py.
if not use_local_cuda_cluster:
    scheduler_address = '10.200.11.12:8786'
    client = Client(address=scheduler_address)

In [4]:
client

Client Scheduler: tcp://10.200.11.12:8786 Dashboard: http://10.200.11.12:8787/status,Cluster Workers: 32 Cores: 160 Memory: 2.20 TB


In [5]:
%%time
# Restart all workers. This also clears GPU memory.
client.restart()

distributed.client - ERROR - Restart timed out after 20.000000 seconds


CPU times: user 22.8 ms, sys: 0 ns, total: 22.8 ms
Wall time: 20 s


Client Scheduler: tcp://10.200.11.12:8786 Dashboard: http://10.200.11.12:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [65]:
# Below from mortgage/E2E.ipynb.
def load_performance_csv(performance_path, **kwargs):
    """ Loads performance data

    Returns
    -------
    GPU DataFrame
    """
    
    cols = [
        "loan_id", "monthly_reporting_period", "servicer", "interest_rate", "current_actual_upb",
        "loan_age", "remaining_months_to_legal_maturity", "adj_remaining_months_to_maturity",
        "maturity_date", "msa", "current_loan_delinquency_status", "mod_flag", "zero_balance_code",
        "zero_balance_effective_date", "last_paid_installment_date", "foreclosed_after",
        "disposition_date", "foreclosure_costs", "prop_preservation_and_repair_costs",
        "asset_recovery_costs", "misc_holding_expenses", "holding_taxes", "net_sale_proceeds",
        "credit_enhancement_proceeds", "repurchase_make_whole_proceeds", "other_foreclosure_proceeds",
        "non_interest_bearing_upb", "principal_forgiveness_upb", "repurchase_make_whole_proceeds_flag",
        "foreclosure_principal_write_off_amount", "servicing_activity_indicator"
    ]
    
    dtypes = OrderedDict([
        ("loan_id", "int64"),
        ("monthly_reporting_period", "date"),
        ("servicer", "category"),
        ("interest_rate", "float64"),
        ("current_actual_upb", "float64"),
        ("loan_age", "float64"),
        ("remaining_months_to_legal_maturity", "float64"),
        ("adj_remaining_months_to_maturity", "float64"),
        ("maturity_date", "date"),
        ("msa", "float64"),
        ("current_loan_delinquency_status", "int32"),
        ("mod_flag", "category"),
        ("zero_balance_code", "category"),
        ("zero_balance_effective_date", "date"),
        ("last_paid_installment_date", "date"),
        ("foreclosed_after", "date"),
        ("disposition_date", "date"),
        ("foreclosure_costs", "float64"),
        ("prop_preservation_and_repair_costs", "float64"),
        ("asset_recovery_costs", "float64"),
        ("misc_holding_expenses", "float64"),
        ("holding_taxes", "float64"),
        ("net_sale_proceeds", "float64"),
        ("credit_enhancement_proceeds", "float64"),
        ("repurchase_make_whole_proceeds", "float64"),
        ("other_foreclosure_proceeds", "float64"),
        ("non_interest_bearing_upb", "float64"),
        ("principal_forgiveness_upb", "float64"),
        ("repurchase_make_whole_proceeds_flag", "category"),
        ("foreclosure_principal_write_off_amount", "float64"),
        ("servicing_activity_indicator", "category")
    ])
    dtypes = {k: ("str" if v=="date" else v) for k,v in dtypes.items()}

    return dask.dataframe.read_csv(performance_path, names=cols, delimiter='|', dtype=dtypes, blocksize=1e9)

In [66]:
# Identify list of files to load.
data_dir = '/mnt/isilon1/data/mortgage'
perf_file = []
perf_file += glob.glob(data_dir + '/perf/Performance_*')
#perf_file += glob.glob(data_dir + '/perf/Performance_*')
#perf_file += glob.glob(data_dir + '/perf/Performance_2016*.txt')
len(perf_file)

112

In [67]:
%%time
perf_ddf = load_performance_csv(perf_file)

CPU times: user 689 ms, sys: 0 ns, total: 689 ms
Wall time: 698 ms


In [68]:
perf_ddf

,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,current_loan_delinquency_status,mod_flag,zero_balance_code,zero_balance_effective_date,last_paid_installment_date,foreclosed_after,disposition_date,foreclosure_costs,prop_preservation_and_repair_costs,asset_recovery_costs,misc_holding_expenses,holding_taxes,net_sale_proceeds,credit_enhancement_proceeds,repurchase_make_whole_proceeds,other_foreclosure_proceeds,non_interest_bearing_upb,principal_forgiveness_upb,repurchase_make_whole_proceeds_flag,foreclosure_principal_write_off_amount,servicing_activity_indicator
npartitions=246,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,category[unknown],float64,float64,float64,float64,float64,object,float64,int32,category[unknown],category[unknown],object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,category[unknown],float64,category[unknown]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [69]:
%%time
perf_ddf.head()

CPU times: user 51.1 ms, sys: 2.79 ms, total: 53.9 ms
Wall time: 32.8 s


,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,holding_taxes,net_sale_proceeds,credit_enhancement_proceeds,repurchase_make_whole_proceeds,other_foreclosure_proceeds,non_interest_bearing_upb,principal_forgiveness_upb,repurchase_make_whole_proceeds_flag,foreclosure_principal_write_off_amount,servicing_activity_indicator
0,548637365156,01/01/2005,NaN,5.75,130964.67,9.0,351.0,351.0,04/2034,35840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,548637365156,01/01/2006,NaN,5.75,129125.98,21.0,339.0,338.0,04/2034,35840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,548637365156,01/01/2007,NaN,5.75,127061.77,33.0,327.0,325.0,04/2034,35840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,548637365156,02/01/2005,NaN,5.75,130820.14,10.0,350.0,350.0,04/2034,35840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,548637365156,02/01/2006,NaN,5.75,128972.64,22.0,338.0,337.0,04/2034,35840.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
to_write_ddf = perf_ddf.repartition(npartitions=16*6)
to_write_ddf

,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,current_loan_delinquency_status,mod_flag,zero_balance_code,zero_balance_effective_date,last_paid_installment_date,foreclosed_after,disposition_date,foreclosure_costs,prop_preservation_and_repair_costs,asset_recovery_costs,misc_holding_expenses,holding_taxes,net_sale_proceeds,credit_enhancement_proceeds,repurchase_make_whole_proceeds,other_foreclosure_proceeds,non_interest_bearing_upb,principal_forgiveness_upb,repurchase_make_whole_proceeds_flag,foreclosure_principal_write_off_amount,servicing_activity_indicator
npartitions=96,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,category[unknown],float64,float64,float64,float64,float64,object,float64,int32,category[unknown],category[unknown],object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,category[unknown],float64,category[unknown]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [75]:
import snappy

ModuleNotFoundError: No module named 'snappy'

In [74]:
p = to_write_ddf.get_partition(0).to
p

,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,current_loan_delinquency_status,mod_flag,zero_balance_code,zero_balance_effective_date,last_paid_installment_date,foreclosed_after,disposition_date,foreclosure_costs,prop_preservation_and_repair_costs,asset_recovery_costs,misc_holding_expenses,holding_taxes,net_sale_proceeds,credit_enhancement_proceeds,repurchase_make_whole_proceeds,other_foreclosure_proceeds,non_interest_bearing_upb,principal_forgiveness_upb,repurchase_make_whole_proceeds_flag,foreclosure_principal_write_off_amount,servicing_activity_indicator
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,category[unknown],float64,float64,float64,float64,float64,object,float64,int32,category[unknown],category[unknown],object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,category[unknown],float64,category[unknown]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [72]:
def to_orc(
    df,
    path,
    compression=None,
    compute=True,
):
    """Store Dask.dataframe to ORC files

    Notes
    -----
    Each partition will be written to a separate file.

    Parameters
    ----------
    df : dask.dataframe.DataFrame
    path : string or pathlib.Path
        Destination directory for data.  Prepend with protocol like ``s3://``
        or ``hdfs://`` for remote data.
    compute : bool, optional
        If True (default) then the result is computed immediately. If False
        then a ``dask.delayed`` object is returned for future computation.
    """
    os.mkdir(path)    
    filenames = [os.path.join(path, "part.%i.orc" % i) for i in range(df.npartitions)]    
    parts = [
        dask.delayed(d.to_orc)(filename)
        for d, filename in zip(df.to_delayed(), filenames)
    ]
    out = dask.delayed(lambda x: None)(parts)    
    if compute:
        out = out.compute()
    return out

In [73]:
orc_dir = os.path.join(data_dir, 'perf.orc')
if os.path.exists(orc_dir): shutil.rmtree(orc_dir)
to_orc(to_write_ddf, orc_dir)

AttributeError: 'DataFrame' object has no attribute 'to_orc'